# OpenSearch 한글 형태소 분석기 통한 키워드 검색 
>이 노트북은,
> - SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.m5.large 인스턴스에서 테스트 되었습니다.
> - SageMaker Notebook **`conda_python3`** 에서 테스트 되었습니다.


여기서는 OpenSearch 가 설치된 것을 가정하고, 한글 형태소 분석기의 사용하는 법을 알려 드립니다.

---

### [중요]
- 이 노트북은 Bedrock Titan Embedding Model 을 기본으로 사용합니다. 
- 오픈 서치 서비스가 액티브 된 상태를 가정 합니다.
- 앞서 02_OpenSearch_setup 노트북의 clean-up 이전 부분까지 모두 완료가 되어 있어야 합니다.


---
## Ref: 
- [Amazon OpenSearch Service로 검색 구현하기](https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/blog-reindex)
- [OpenSearch Python Client](https://opensearch.org/docs/1.3/clients/python-high-level/)
- [OpenSearch Match, Multi-Match, and Match Phrase Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-match-multi-match-and-match-phrase-queries/)
- OpenSearch Query 에서 Filter, Must, Should, Not Mush 에 대한 설명 입니다.
    - [OpenSearch Boolean Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-boolean-queries/#:~:text=Boolean%20queries%20are%20used%20to,as%20terms%2C%20match%20and%20query_string.)
- [OpenSearch Query Description (한글)](https://esbook.kimjmin.net/05-search)


## 1. 환경 세팅

In [1]:
import boto3
region = boto3.Session().region_name
opensearch = boto3.client('opensearch', region)

%store -r opensearch_user_id opensearch_user_password domain_name opensearch_domain_endpoint

try:
    opensearch_user_id
    opensearch_user_password
    domain_name
    opensearch_domain_endpoint
   
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

no stored variable or alias opensearch_user_id
no stored variable or alias opensearch_user_password
no stored variable or alias domain_name
no stored variable or alias opensearch_domain_endpoint
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# OpenSearch 접속 정보 지정

# [필수] 아래 OpenSearch 정보는 각자 환경에 맞게 수정 합니다.

opensearch_user_id = 'raguser'
opensearch_user_password = 'Passw0rd1!'

domain_name = 'jesamkim-opensearch-rag'
opensearch_domain_endpoint = 'https://search-jesamkim-opensearch-rag-5wbkv7qjrlci47h5ka63cw5fxy.us-west-2.es.amazonaws.com'

### Bedrock Client 생성

In [4]:
import boto3
import os
import json
from botocore.config import Config
import botocore 
from pprint import pprint
from termcolor import colored

session = boto3.Session()

retry_config = Config(
    region_name=os.environ.get("AWS_DEFAULT_REGION", None),
    retries={
        "max_attempts": 10,
        "mode": "standard",
    },
)

# modelId = "anthropic.claude-instant-v1"  # (Change this to try different model versions)
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = "application/json"
contentType = "application/json"

bedrock = boto3.client(service_name='bedrock')
boto3_bedrock = boto3.client(service_name='bedrock-runtime',config=retry_config)

model_list = bedrock.list_foundation_models()
result = [(fm_list["modelName"], fm_list["modelId"]) for fm_list in model_list["modelSummaries"] if fm_list['inferenceTypesSupported'] == ['ON_DEMAND']]
pprint(result)

[('Titan Text Large', 'amazon.titan-tg1-large'),
 ('Titan Text Embeddings v2', 'amazon.titan-embed-g1-text-02'),
 ('Titan Text G1 - Lite', 'amazon.titan-text-lite-v1'),
 ('Titan Text G1 - Express', 'amazon.titan-text-express-v1'),
 ('Titan Embeddings G1 - Text', 'amazon.titan-embed-text-v1'),
 ('Titan Multimodal Embeddings G1', 'amazon.titan-embed-image-v1'),
 ('Titan Image Generator G1', 'amazon.titan-image-generator-v1'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl-v0'),
 ('SDXL 1.0', 'stability.stable-diffusion-xl-v1'),
 ('J2 Grande Instruct', 'ai21.j2-grande-instruct'),
 ('J2 Jumbo Instruct', 'ai21.j2-jumbo-instruct'),
 ('Jurassic-2 Mid', 'ai21.j2-mid'),
 ('Jurassic-2 Mid', 'ai21.j2-mid-v1'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra-v1'),
 ('Claude Instant', 'anthropic.claude-instant-v1'),
 ('Claude', 'anthropic.claude-v2:1'),
 ('Claude', 'anthropic.claude-v2'),
 ('Claude 3 Sonnet', 'anthropic.clau

## 2. Titan Embedding 및 LLM 인 Claude-3 sonnet 모델 로딩

### LLM 로딩 (Claude-v3 sonnet)

In [5]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
llm_text = BedrockChat(
    model_id=modelId,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature" : 0,
        "top_k": 0,
        "top_p": 0.0
    }
)
llm_text

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import ChatBedrock`.
  warn_deprecated(


BedrockChat(client=<botocore.client.BedrockRuntime object at 0x7fbb9984b970>, region_name='us-west-2', model_id='anthropic.claude-3-sonnet-20240229-v1:0', model_kwargs={'anthropic_version': 'bedrock-2023-05-31', 'max_tokens': 4096, 'temperature': 0, 'top_k': 0, 'top_p': 0.0}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7fbb9a788430>])

In [7]:
prompt1 = "나는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요."
messages = [
    HumanMessage(content=prompt1)
]

# messages = [
#     {"role": "user", "content": [{"type": "text", "text": prompt1}]},
# ]

response1 = llm_text.invoke(messages)

생명보험과 손해보험은 보험의 주요 유형으로 다음과 같은 차이점이 있습니다.

1. 보장 대상
- 생명보험은 사람의 생명과 관련된 위험을 보장합니다. 예를 들어 사망, 상해, 질병 등을 대상으로 합니다.
- 손해보험은 재산상의 손해나 배상책임을 보장합니다. 예를 들어 화재, 자동차사고, 배상책임 등을 대상으로 합니다.

2. 보험금 지급 사유
- 생명보험은 피보험자의 사망, 상해, 질병 등 인적 위험이 발생했을 때 보험금을 지급합니다.
- 손해보험은 재물의 손해나 제3자에 대한 배상책임이 발생했을 때 실제 입은 손해를 보상합니다.

3. 보험기간
- 생명보험은 일반적으로 장기계약이며, 종신보험의 경우 피보험자 종신까지 보장됩니다.
- 손해보험은 단기계약이 일반적이며, 1년 만기로 갱신하는 경우가 많습니다.

4. 보험료 산정 기준
- 생명보험료는 피보험자의 나이, 건강상태, 가입금액 등에 따라 결정됩니다.
- 손해보험료는 보험목적물의 가액, 위험률, 보상한도 등에 따라 결정됩니다.

요약하면 생명보험은 개인의 생명과 관련된 위험을 장기적으로 보장하고, 손해보험은 재산상 손해나 배상책임을 단기적으로 보상하는 차이가 있습니다.

### Embedding 모델 선택

In [8]:
from langchain_community.embeddings import BedrockEmbeddings

llm_emb = BedrockEmbeddings(
    client=boto3_bedrock,
    # model_id="cohere.embed-multilingual-v3"
    model_id="amazon.titan-embed-g1-text-02"
)

-------------------

## 3. OpenSearch 벡터 Index 생성
### 선수 조건
- 랭체인 오프서처 참고 자료
    - [Langchain Opensearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch)

### 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [9]:
from opensearchpy import OpenSearch, RequestsHttpConnection
http_auth = (opensearch_user_id, opensearch_user_password)
os_client = OpenSearch(
            hosts=[
                {'host': opensearch_domain_endpoint.replace("https://", ""),
                 'port': 443
                }
            ],
            http_auth=http_auth, # Master username, Master password,
            use_ssl=True,
            verify_certs=True,
            connection_class=RequestsHttpConnection
        )

### index 생성

In [10]:
index_name = "index-03"
exists = os_client.indices.exists(index_name)

if exists:
    os_client.indices.delete(index=index_name)
    print("Index is deleted")
else:
    print("Index does not exist")

Index is deleted


In [11]:
## metadata, text, vector_field 의 네이밍은 langchain에서 지정된 이름
### model에 따라 dimension 사이즈 변경 필요 (Titan : 1536, Cohere : 1024)
import json

with open('index_body_simple.json', 'r') as f:
    index_body = json.load(f)

print(json.dumps(index_body, indent=2))


{
  "settings": {
    "index.knn": true,
    "index.knn.algo_param.ef_search": 512
  },
  "mappings": {
    "properties": {
      "metadata": {
        "properties": {
          "source": {
            "type": "keyword"
          },
          "type": {
            "type": "keyword"
          },
          "timestamp": {
            "type": "date"
          }
        }
      },
      "vector_field": {
        "type": "knn_vector",
        "dimension": 1536,
        "method": {
          "engine": "faiss",
          "name": "hnsw",
          "parameters": {
            "ef_construction": 512,
            "m": 16
          },
          "space_type": "l2"
        }
      }
    }
  }
}


In [12]:
os_client.indices.create(index_name, body=index_body)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'index-03'}

In [13]:
%%time
from langchain_community.vectorstores import OpenSearchVectorSearch

vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
)

CPU times: user 7.16 ms, sys: 0 ns, total: 7.16 ms
Wall time: 6.83 ms


## 4. 데이터 준비


In [14]:
import time
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PyPDFium2Loader


from langchain_core.documents import Document

# from llmsherpa.readers import LayoutPDFReader

In [15]:
import glob

# 오픈서치 인덱스에 저장되는 데이터 pdf 파일 경로
data_path = './data/03/*'

pdf_list = glob.glob(data_path)
pdf_list

['./data/03/국제 신재생에너지 정책변화 및 시장 분석_22-26.pdf']

In [16]:
from multiprocessing.pool import ThreadPool
from multiprocessing import  Manager

import pdfplumber

In [17]:
import re

def prune_text(text, current_pdf_file):

    def replace_cid(match):
        print(f"Please check PDF file {current_pdf_file} : {match}")
        ascii_num = int(match.group(1))
        try:
            return chr(ascii_num)
        except:
            return ''  # In case of conversion failure, return empty string

    # Regular expression to find all (cid:x) patterns
    cid_pattern = re.compile(r'\(cid:(\d+)\)')
    pruned_text = re.sub(cid_pattern, replace_cid, text)
    return pruned_text

In [18]:
from datetime import datetime

def read_pdf(param):
    vector_db = param[0]
    current_pdf_file = param[1]
    print(f"current_pdf_file : {current_pdf_file}")
    docs = []
    source_name = current_pdf_file.split('/')[-1]
    type_name = source_name.split('_')[0]
    
    with pdfplumber.open(current_pdf_file) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                pruned_text = prune_text(page_text, current_pdf_file)
            else:
                pruned_text = ""
            if len(pruned_text) >= 20:  ## 임의로 20 이상인 sentence만 뽑도록 함
                chunk = Document(
                    page_content=pruned_text.replace('\n',' '),
                    metadata={
                        "source" : source_name,
                        "type": type_name,
                        "timestamp": datetime.now()
                    }
                )
                #print(f"chunk : {chunk}")
                docs.append(chunk)
    if len(docs) > 0 :
        vector_db.add_documents(docs)

In [19]:
manager = Manager()
result_dict = manager.dict()

# ml.m5.xlarge에서 multiprocessing으로 동작 확인
param = [(vector_db, current_pdf_file)for current_pdf_file in pdf_list]

num_processes = len(pdf_list)%os.cpu_count()

if num_processes == 0 :
    num_processes = os.cpu_count() - 1

print(f"num of process : {num_processes}")

with ThreadPool(processes=num_processes) as pool:
    pool.map(read_pdf, param)
    pool.close()
    pool.join()

num of process : 1
current_pdf_file : ./data/03/국제 신재생에너지 정책변화 및 시장 분석_22-26.pdf


### OpenSearch에 생성된 인덱스의 구성 확인

In [20]:
index_info = os_client.indices.get(index=index_name)
print(json.dumps(index_info, indent=2))

{
  "index-03": {
    "aliases": {},
    "mappings": {
      "properties": {
        "metadata": {
          "properties": {
            "source": {
              "type": "keyword"
            },
            "timestamp": {
              "type": "date"
            },
            "type": {
              "type": "keyword"
            }
          }
        },
        "text": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "vector_field": {
          "type": "knn_vector",
          "dimension": 1536,
          "method": {
            "engine": "faiss",
            "space_type": "l2",
            "name": "hnsw",
            "parameters": {
              "ef_construction": 512,
              "m": 16
            }
          }
        }
      }
    },
    "settings": {
      "index": {
        "replication": {
          "type": "DOCUMENT"
        },
        

## 5. 어휘를 기반으로 한 전문(full-text) 검색 - Lexical Search

> query의 문장은 Nori 형태소 분석기에 의해 형태소로 분리됨
> OpenSearch에 입력된 (PDF에서 추출한) 문장(청크)들과 BM25 알고리즘을 기반으로 가장 유사한(score가 높은) 값부터 상위 (k=5)개의 결과가 나타남  

In [21]:
from opensearch_dsl import Search

In [22]:
def result_to_dataframe(response):
    import pandas as pd

    pd.set_option('display.max_columns', 150)
    pd.set_option('display.max_colwidth', None)

    result = []
    for res in response['hits']['hits']:
        # print(res.keys())
        result.append([res['_index'], round(res['_score'], 4), res['_source']['metadata']['type'], res['_source']['text']])
    df = pd.DataFrame(result, columns=['index_name', 'score', 'type', 'text'])
    return df.style.set_properties(**{'text-align': 'left'})

In [23]:
def query_lexical(query, filter=[], k=5):
    QUERY_TEMPLATE = {
        "size": k,
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": query
                        }
                    }
                ],
                "filter": filter
            }
        }
    }
    if len(filter) > 0:
        QUERY_TEMPLATE["query"]["bool"]["filter"].extend(filter)
    return QUERY_TEMPLATE

In [24]:
query = "2020년 전세계 에너지 공급량은 얼마인가요?"

response_lexical_only = os_client.search(
    body=query_lexical(query),
    index=index_name
)

time_took_lexical_only = response_lexical_only['took']
print('검색에 걸린 시간: ', time_took_lexical_only, 'ms')

print("<<사용자 입력 쿼리 문장>>: ", query)

result_to_dataframe(response_lexical_only)

검색에 걸린 시간:  5 ms
<<사용자 입력 쿼리 문장>>:  2020년 전세계 에너지 공급량은 얼마인가요?


,index_name,score,type,text
0,index-03,11.883800,국제 신재생에너지 정책변화 및 시장 분석,"제2장 국제 재생에너지 시장 동향 제2장 국제 재생에너지 시장 동향 본 연구의 주요 목적 중 하나는 매년 변화하는 재생에너지 시장 동향을 살펴보는 것으로, 자체적으로 수행한 데이터 분석 결과와 IEA, REN21, Enerdtata, BNEF, IRENA 등 주요 국제기구에서 발간하는 자료를 토대로 재생에너지 시장 변화를 살 펴보고자 한다. 먼저 1절에서는 전세계 관점에서 재생에너지 보급, 산업, 가격, 전망 에 대해 다루고, 2절에서는 국가별 재생에너지 보급 및 가격 현황을 살펴보았다. 1. 재생에너지 보급 동향 1.1. 1차에너지1) 2020년 전세계 에너지 공급량은 14,177Mtoe로 이 중 재생에너지 공급량은 14.6%에 해당하는 2,077Mtoe였다. 비중이 가장 높은 에너지원은 석유로 30.0%를 차지하였고, 다음으로 석탄(26.9%), 천연가스(23.5%), 순이다. 2010년과 비교하여 재생에너지 공급량은 508Mtoe 증가하였으며, 1차에너지에서 재생에너지 비중은 1) 여기서는 Enerdata(최종접속일: 2022.11.20)에서 제공하는 자료를 활용하여 작성하였음. 5"
1,index-03,5.876000,국제 신재생에너지 정책변화 및 시장 분석,"2.7%p 증가하였다([그림 2-1]). 재생에너지 원별로 나누어보면 2020년 기준 바이 오매스 비중이 65.1%로 가장 높고, 다음으로 수력(17.9%), 풍력(6.7%), 지열(5.3%) 태양광(3.4%), 태양열(1.6%) 순이다. 바이오매스 비중이 높은 이유는 개발도상국에 서 난방과 취사용으로 주로 활용하기 때문이다.2) 2010년과 비교하여 공급량은 바 이오매스가 191Mtoe로 가장 많이 증가하였고, 다음으로 풍력(109Mtoe), 수력 (76Mtoe), 태양광(67Mtoe) 순이다. 하지만 비중은 풍력이 4.8%p로 가장 많이 증 가하였고, 다음으로 태양광(3.2%), 지열(1.2%) 순이며, 바이오매스와 수력은 각각 8.9%p와 1.0%p 감소하였다([그림 2-2]). [그림 2-1] 에너지원별 1차에너지 공급(TPES) 비중 <2020년> <2010년> 자료: Enerdata(최종접속일: 2022.11.20)의 자료를 활용하여 저자 작성 [그림 2-2] 재생에너지 원별 1차에너지 공급(TPES) 비중 <2010년> <2020년> 자료: Enerdata(최종접속일: 2022.11.20)의 자료를 활용하여 저자 작성 2) 공지영·조일현(2021), p.4. 6"
2,index-03,5.587500,국제 신재생에너지 정책변화 및 시장 분석,"제2장 국제 재생에너지 시장 동향 2. 국가별 재생에너지 현황 비교 2.1. 국가별 재생에너지 보급 현황 [그림 2-17]은 OECD 국가별 재생에너지 발전 비중을 나타낸다. 2020년 우리나 라 재생에너지 발전 비중은 4.7% 수준으로 37위이며, OECD 평균(27.3%)과 전세계 평균(26.1%)보다 낮게 나타났다.25) 전세계 평균(26.1%)에 도달하지 못한 OECD 국 가는 우리나라를 포함하여 모두 13개국이다. OECD 중에서 재생에너지 발전 비중이 가장 높은 나라는 코스타리카(99.2%), 아이슬란드(99.1%), 노르웨이(96.1%) 순이다. 상위 10개 국가 중 덴마크를 제외한 나머지 국가는 주 에너지원이 수력이라는 특징 이 있으며, 덴마크의 주 에너지원은 풍력이다. 10위인 스웨덴의 수력 비중은 44.2%, 원자력 비중은 30.1%로 원자력 비중도 상당히 높은 것이 특징이다. 세계 각국은 자원 및 경제 상황에 따라 에너지원을 다양하게 사용하는 가운데, 영 국, 독일, 호주, 미국, 일본 등은 탄소중립 달성을 위해 에너지전환 정책을 적극적으 로 추진 중이다. 영국은 석탄 감축 정책을 펼치면서 재생에너지 보급을 빠르게 증가 시킨 나라 중 하나로, 원별로는 천연가스 비중이 36.5% 가장 높고, 다음으로 풍력 (24.2%), 원자력(16.1%), 바이오연료(11.2%) 등의 비중이 높다. 영국의 2020년 석 탄 발전 비중은 2.0% 수준으로 2015년 22.7%와 비교하여 대폭 감소하였다. 독일 은 탈원전·탈석탄을 추진 중인 국가이지만 그 결과는 미진한데, 원별 비중을 보면 석탄이 25.5%로 가장 높고, 다음으로 풍력(22.5%), 천연가스(17.1%), 원자력 (11.1%), 태양광(8.7%) 순이다. 호주는 주요 석탄 생산국으로, 원별 비중을 보면 석 탄이 54.9%로 가장 높고, 다음으로 천연가스(20.8%), 태양광(7.9%), 풍력(7.7%), 수력(5.7%) 순이다. 미국은 바이든 정부 이후 친환경 정책을 강조하는 국가로, 원별 비중을 보면 천연가스 비중이 39.3%로 가장 높고, 다음으로 석탄(20.0%), 원자력 (19.4%), 풍력(8.0%), 수력(7.4%), 태양광(2.7%) 순이다. 일본은 후쿠시마 사건 이 후 원자력 발전이 줄어들면서 대체에너지로 재생에너지 보급 추진 중인 국가로, 원 별 비중을 보면 천연가스가 37.7%로 가장 높고, 다음으로 석탄(30.4%), 수력 (8.6%), 태양광(7.6%) 순이다. 특히 일본은 에너지 섬이면서 화석연료 수입 비중이 25)OECD iLibrary홈페이지(최종접속일: 2022.9.8) 데이터 참고함. 23"
3,index-03,5.250500,국제 신재생에너지 정책변화 및 시장 분석,"높다는 점에서 우리나라와 에너지 환경은 비슷하지만, 재생에너지 비중은 우리나보 다 4배 가까이 높다. IEA 기준 우리나라의 원별 비중을 보면 석탄이 38.7%로 가장 높고, 다음으로 원자력(27.3%), 천연가스(25.8%), 태양광(3.1%), 바이오연료(1.4%) 순이다.26) [그림 2-17] OECD 국가별 재생에너지 발전 비중(2020년) 자료: OECD iLibrary홈페이지(최종접속일: 2022.9.8) 자료 활용하여 저자 작성 본고에서는 추가로 Enerdata 자료를 활용하여 전세계 186개국을 대상으로 재생에 너지 보급 수준을 비교하였다. 이를 위해 2020년 발전량 자료를 활용하였고, 사분위 수를 도출하여 국가들을 그룹화하였다. 발전량에 따른 국가들의 지리적 분포도는 [그 림 2-18]~[그림 2-21]에 제시하였다. 개별 국가가 포함된 그룹 순위에 따라 다른 색 으로 지도에 표시되어 있으며 보급 수준이 가장 높은 상위 25% 그룹(Q4)이 가장 짙 은 색으로, 보급 수준이 낮은 하위 50% 그룹(Q2, Q1)이 가장 흐린 색으로 표시되어 총 3개의 그룹을 보여준다. 26)OECD iLibrary홈페이지(최종접속일: 2022.9.8) 데이터를 참고함. 24"
4,index-03,4.405900,국제 신재생에너지 정책변화 및 시장 분석,"은 원자재 수출을 차단하고, 러시아 화폐인 루블화 결제를 강제하면서 오히려 유럽 의 심각한 에너지 수급난을 불러일으켰다.38)39) 결과적으로는 전세계 원유, 천연가 스 가격을 상승시켰을 뿐만 아니라 대체제인 석탄 가격의 수요 증가와 EU의 러시아 산 석탄수입 금지 제재안이 통과되면서 화석연료 가격이 폭등하였다.40) [그림 3-2] 국제 화석연료 가격 추이 자료: 에너지경제연구원(2022b), p.6. 러·우 전쟁이 전세계 에너지 부문에 경종을 울렸다는 사실은 부인할 수 없다. 현 재의 위기는 각국의 러시아 가스에 대한 의존이 과도함을 상기시키며 타국에 대한 높은 에너지공급 의존도는 국제 정세에 따라 에너지 안보에 약점이 될 수 있음을 여 실히 보여준다. 또한, 러·우 전쟁을 통해 화석 연료에 기반한 에너지 시스템이 국제 위기의 영향에 매우 취약하다는 것을 확인할 수 있다. 이와 달리 재생에너지원을 기 반으로 하는 시스템은 상대적으로 외부 환경에 대해 독립적이며, 지속가능하고 탄력 적이며, 안정적이다. 그렇기에 유럽을 비롯한 여러 국가에서 재생가능한 에너지원으 로 전환함으로써 에너지 안보 확립과 더불어 전세계 기후 위기 대처라는 더 크고 장 기적인 목표를 함께 달성하고자 한다. EU는 2022년 4월 7일부로 러시아산 석탄에 대하여 8월 이후 신규계약을 금지하는 완전한

### 결과값에 매칭되는 Term 확인

BM25 알고리즘에 의해 스코어링이 되고, PDF에서 추출된 문장(chunk)에서 어떤 term들과 매칭되었는지를 확인하려면 아래의 코드를 실행합니다. 아래 쿼리는 결과값에 매칭되는 term을 강조할 수 있도록 html 태그를 추가합니다.

In [25]:
def query_lexical_with_highlight(query, filter=[], k=5):
    QUERY_TEMPLATE = {
        "size": k,
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": query
                        }
                    }
                ],
                "filter": filter
            }
        },
        "highlight": {
            "pre_tags": [
                "<span style='color:red'>"
            ],
            "post_tags": [
                "</span>"
            ],
            "fields": {
                "text": {}
            }
        }
    }
    if len(filter) > 0:
        QUERY_TEMPLATE["query"]["bool"]["filter"].extend(filter)
    return QUERY_TEMPLATE

response_lexical_with_highlight = os_client.search(
    body=query_lexical_with_highlight(query),
    index=index_name
)


In [26]:
from IPython.display import HTML

import pandas as pd
temp_arr = []

for res in response_lexical_with_highlight['hits']['hits']:
            # result.append([res['_index'], round(res['_score'], 4), res['_source']['metadata']['type'], res['_source']['text']])

    temp_arr.append([res['_score'], res['highlight']['text']])

# print("---------- html 태그가 포함된 결과 ------------")
# print(temp_arr)
# print("-------------------------------------------")
    
#df = pd.DataFrame(temp_arr)
df = pd.DataFrame(temp_arr, columns=['score', '각 문서(chuck) 내에서 매칭된 부분'])

print("<<사용자 입력 쿼리 문장>>: ", query)

HTML(df.to_html(escape=False))


<<사용자 입력 쿼리 문장>>:  2020년 전세계 에너지 공급량은 얼마인가요?


,score,각 문서(chuck) 내에서 매칭된 부분
0,11.883775,"[먼저 1절에서는 전세계 관점에서 재생에너지 보급, 산업, 가격, 전망 에 대해 다루고, 2절에서는 국가별 재생에너지 보급 및 가격 현황을 살펴보았다. 1., 재생에너지 보급 동향 1.1. 1차에너지1) 2020년 전세계 에너지 공급량은 14,177Mtoe로 이 중 재생에너지 공급량은 14.6%에 해당하는 2,077Mtoe였다., 비중이 가장 높은 에너지원은 석유로 30.0%를 차지하였고, 다음으로 석탄(26.9%), 천연가스(23.5%), 순이다. 2010년과 비교하여 재생에너지 공급량은 508Mtoe 증가하였으며]"
1,5.875968,"[재생에너지 원별로 나누어보면 2020년 기준 바이 오매스 비중이 65.1%로 가장 높고, 다음으로 수력(17.9%), 풍력(6.7%), 지열(5.3%) 태양광(3.4%), 태양열(1.6%, 바이오매스 비중이 높은 이유는 개발도상국에 서 난방과 취사용으로 주로 활용하기 때문이다.2) 2010년과 비교하여 공급량은 바 이오매스가 191Mtoe로 가장 많이 증가하였고, 다음으로, [그림 2-1] 에너지원별 1차에너지 공급(TPES) 비중 <2020년> <2010년> 자료: Enerdata(최종접속일: 2022.11.20)의 자료를 활용하여 저자 작성 [그림, 2-2] 재생에너지 원별 1차에너지 공급(TPES) 비중 <2010년> <2020년> 자료: Enerdata(최종접속일: 2022.11.20)의 자료를 활용하여 저자 작성 2) 공지영]"
2,5.587486,"[국가별 재생에너지 보급 현황 [그림 2-17]은 OECD 국가별 재생에너지 발전 비중을 나타낸다. 2020년 우리나 라 재생에너지 발전 비중은 4.7% 수준으로 37위이며, OECD, 평균(27.3%)과 전세계 평균(26.1%)보다 낮게 나타났다.25) 전세계 평균(26.1%)에 도달하지 못한 OECD 국 가는 우리나라를 포함하여 모두 13개국이다., 영국의 2020년 석 탄 발전 비중은 2.0% 수준으로 2015년 22.7%와 비교하여 대폭 감소하였다., 특히 일본은 에너지 섬이면서 화석연료 수입 비중이 25)OECD iLibrary홈페이지(최종접속일: 2022.9.8) 데이터 참고함. 23]"
3,5.250500,"[높다는 점에서 우리나라와 에너지 환경은 비슷하지만, 재생에너지 비중은 우리나보 다 4배 가까이 높다., 가장 높고, 다음으로 원자력(27.3%), 천연가스(25.8%), 태양광(3.1%), 바이오연료(1.4%) 순이다.26) [그림 2-17] OECD 국가별 재생에너지 발전 비중(2020년, ) 자료: OECD iLibrary홈페이지(최종접속일: 2022.9.8) 자료 활용하여 저자 작성 본고에서는 추가로 Enerdata 자료를 활용하여 전세계 186개국을 대상으로 재생에, 이를 위해 2020년 발전량 자료를 활용하였고, 사분위 수를 도출하여 국가들을 그룹화하였다.]"
4,4.405927,"[은 원자재 수출을 차단하고, 러시아 화폐인 루블화 결제를 강제하면서 오히려 유럽 의 심각한 에너지 수급난을 불러일으켰다.38)39) 결과적으로는 전세계 원유, 천연가 스 가격을 상승시켰을, 러·우 전쟁이 전세계 에너지 부문에 경종을 울렸다는 사실은 부인할 수 없다., 현 재의 위기는 각국의 러시아 가스에 대한 의존이 과도함을 상기시키며 타국에 대한 높은 에너지공급 의존도는 국제 정세에 따라 에너지 안보에 약점이 될 수 있음을 여 실히 보여준다., 또한, 러·우 전쟁을 통해 화석 연료에 기반한 에너지 시스템이 국제 위기의 영향에 매우 취약하다는 것을 확인할 수 있다., 그렇기에 유럽을 비롯한 여러 국가에서 재생가능한 에너지원으 로 전환함으로써 에너지 안보 확립과 더불어 전세계 기후 위기 대처라는 더 크고 장 기적인 목표를 함께 달성하고자 한다.]"


참고> hightlight 요청은 모든 결과를 return하지 않고, 매칭되는 부분만 중점적으로 보여줍니다. 위의 코드에서는 검색에 매칭되는 term의 앞 뒤로 \<span style='color:red'>과 \</span>태그로 눈에 들어오도록 표기했습니다.

## 6. Filter 활용
- document내 metadata를 활용하여 search space를 줄일 수 있다.
- 특히 filter의 경우 search 전에 수행되기 때문에, 검색 속도 향상을 기대할 수 있다
- syntax
    - filter=[{"term"**[고정]**: {"metadata.source"**[메타데이터 이름, 혹은 메타데이터 아니여도 상관없음]**: "신한은행"**[조건명]**}},]
    - list 형식으로 복수개 filter 설정 가능

In [27]:
filter = [
    {"term": {"metadata.source": "국제 신재생에너지 정책변화 및 시장 분석_22-26.pdf"}}
]

response = os_client.search(
    body=query_lexical(query, filter),
    index=index_name
)
result_to_dataframe(response)

,index_name,score,type,text
0,index-03,11.883800,국제 신재생에너지 정책변화 및 시장 분석,"제2장 국제 재생에너지 시장 동향 제2장 국제 재생에너지 시장 동향 본 연구의 주요 목적 중 하나는 매년 변화하는 재생에너지 시장 동향을 살펴보는 것으로, 자체적으로 수행한 데이터 분석 결과와 IEA, REN21, Enerdtata, BNEF, IRENA 등 주요 국제기구에서 발간하는 자료를 토대로 재생에너지 시장 변화를 살 펴보고자 한다. 먼저 1절에서는 전세계 관점에서 재생에너지 보급, 산업, 가격, 전망 에 대해 다루고, 2절에서는 국가별 재생에너지 보급 및 가격 현황을 살펴보았다. 1. 재생에너지 보급 동향 1.1. 1차에너지1) 2020년 전세계 에너지 공급량은 14,177Mtoe로 이 중 재생에너지 공급량은 14.6%에 해당하는 2,077Mtoe였다. 비중이 가장 높은 에너지원은 석유로 30.0%를 차지하였고, 다음으로 석탄(26.9%), 천연가스(23.5%), 순이다. 2010년과 비교하여 재생에너지 공급량은 508Mtoe 증가하였으며, 1차에너지에서 재생에너지 비중은 1) 여기서는 Enerdata(최종접속일: 2022.11.20)에서 제공하는 자료를 활용하여 작성하였음. 5"
1,index-03,5.876000,국제 신재생에너지 정책변화 및 시장 분석,"2.7%p 증가하였다([그림 2-1]). 재생에너지 원별로 나누어보면 2020년 기준 바이 오매스 비중이 65.1%로 가장 높고, 다음으로 수력(17.9%), 풍력(6.7%), 지열(5.3%) 태양광(3.4%), 태양열(1.6%) 순이다. 바이오매스 비중이 높은 이유는 개발도상국에 서 난방과 취사용으로 주로 활용하기 때문이다.2) 2010년과 비교하여 공급량은 바 이오매스가 191Mtoe로 가장 많이 증가하였고, 다음으로 풍력(109Mtoe), 수력 (76Mtoe), 태양광(67Mtoe) 순이다. 하지만 비중은 풍력이 4.8%p로 가장 많이 증 가하였고, 다음으로 태양광(3.2%), 지열(1.2%) 순이며, 바이오매스와 수력은 각각 8.9%p와 1.0%p 감소하였다([그림 2-2]). [그림 2-1] 에너지원별 1차에너지 공급(TPES) 비중 <2020년> <2010년> 자료: Enerdata(최종접속일: 2022.11.20)의 자료를 활용하여 저자 작성 [그림 2-2] 재생에너지 원별 1차에너지 공급(TPES) 비중 <2010년> <2020년> 자료: Enerdata(최종접속일: 2022.11.20)의 자료를 활용하여 저자 작성 2) 공지영·조일현(2021), p.4. 6"
2,index-03,5.587500,국제 신재생에너지 정책변화 및 시장 분석,"제2장 국제 재생에너지 시장 동향 2. 국가별 재생에너지 현황 비교 2.1. 국가별 재생에너지 보급 현황 [그림 2-17]은 OECD 국가별 재생에너지 발전 비중을 나타낸다. 2020년 우리나 라 재생에너지 발전 비중은 4.7% 수준으로 37위이며, OECD 평균(27.3%)과 전세계 평균(26.1%)보다 낮게 나타났다.25) 전세계 평균(26.1%)에 도달하지 못한 OECD 국 가는 우리나라를 포함하여 모두 13개국이다. OECD 중에서 재생에너지 발전 비중이 가장 높은 나라는 코스타리카(99.2%), 아이슬란드(99.1%), 노르웨이(96.1%) 순이다. 상위 10개 국가 중 덴마크를 제외한 나머지 국가는 주 에너지원이 수력이라는 특징 이 있으며, 덴마크의 주 에너지원은 풍력이다. 10위인 스웨덴의 수력 비중은 44.2%, 원자력 비중은 30.1%로 원자력 비중도 상당히 높은 것이 특징이다. 세계 각국은 자원 및 경제 상황에 따라 에너지원을 다양하게 사용하는 가운데, 영 국, 독일, 호주, 미국, 일본 등은 탄소중립 달성을 위해 에너지전환 정책을 적극적으 로 추진 중이다. 영국은 석탄 감축 정책을 펼치면서 재생에너지 보급을 빠르게 증가 시킨 나라 중 하나로, 원별로는 천연가스 비중이 36.5% 가장 높고, 다음으로 풍력 (24.2%), 원자력(16.1%), 바이오연료(11.2%) 등의 비중이 높다. 영국의 2020년 석 탄 발전 비중은 2.0% 수준으로 2015년 22.7%와 비교하여 대폭 감소하였다. 독일 은 탈원전·탈석탄을 추진 중인 국가이지만 그 결과는 미진한데, 원별 비중을 보면 석탄이 25.5%로 가장 높고, 다음으로 풍력(22.5%), 천연가스(17.1%), 원자력 (11.1%), 태양광(8.7%) 순이다. 호주는 주요 석탄 생산국으로, 원별 비중을 보면 석 탄이 54.9%로 가장 높고, 다음으로 천연가스(20.8%), 태양광(7.9%), 풍력(7.7%), 수력(5.7%) 순이다. 미국은 바이든 정부 이후 친환경 정책을 강조하는 국가로, 원별 비중을 보면 천연가스 비중이 39.3%로 가장 높고, 다음으로 석탄(20.0%), 원자력 (19.4%), 풍력(8.0%), 수력(7.4%), 태양광(2.7%) 순이다. 일본은 후쿠시마 사건 이 후 원자력 발전이 줄어들면서 대체에너지로 재생에너지 보급 추진 중인 국가로, 원 별 비중을 보면 천연가스가 37.7%로 가장 높고, 다음으로 석탄(30.4%), 수력 (8.6%), 태양광(7.6%) 순이다. 특히 일본은 에너지 섬이면서 화석연료 수입 비중이 25)OECD iLibrary홈페이지(최종접속일: 2022.9.8) 데이터 참고함. 23"
3,index-03,5.250500,국제 신재생에너지 정책변화 및 시장 분석,"높다는 점에서 우리나라와 에너지 환경은 비슷하지만, 재생에너지 비중은 우리나보 다 4배 가까이 높다. IEA 기준 우리나라의 원별 비중을 보면 석탄이 38.7%로 가장 높고, 다음으로 원자력(27.3%), 천연가스(25.8%), 태양광(3.1%), 바이오연료(1.4%) 순이다.26) [그림 2-17] OECD 국가별 재생에너지 발전 비중(2020년) 자료: OECD iLibrary홈페이지(최종접속일: 2022.9.8) 자료 활용하여 저자 작성 본고에서는 추가로 Enerdata 자료를 활용하여 전세계 186개국을 대상으로 재생에 너지 보급 수준을 비교하였다. 이를 위해 2020년 발전량 자료를 활용하였고, 사분위 수를 도출하여 국가들을 그룹화하였다. 발전량에 따른 국가들의 지리적 분포도는 [그 림 2-18]~[그림 2-21]에 제시하였다. 개별 국가가 포함된 그룹 순위에 따라 다른 색 으로 지도에 표시되어 있으며 보급 수준이 가장 높은 상위 25% 그룹(Q4)이 가장 짙 은 색으로, 보급 수준이 낮은 하위 50% 그룹(Q2, Q1)이 가장 흐린 색으로 표시되어 총 3개의 그룹을 보여준다. 26)OECD iLibrary홈페이지(최종접속일: 2022.9.8) 데이터를 참고함. 24"
4,index-03,4.405900,국제 신재생에너지 정책변화 및 시장 분석,"은 원자재 수출을 차단하고, 러시아 화폐인 루블화 결제를 강제하면서 오히려 유럽 의 심각한 에너지 수급난을 불러일으켰다.38)39) 결과적으로는 전세계 원유, 천연가 스 가격을 상승시켰을 뿐만 아니라 대체제인 석탄 가격의 수요 증가와 EU의 러시아 산 석탄수입 금지 제재안이 통과되면서 화석연료 가격이 폭등하였다.40) [그림 3-2] 국제 화석연료 가격 추이 자료: 에너지경제연구원(2022b), p.6. 러·우 전쟁이 전세계 에너지 부문에 경종을 울렸다는 사실은 부인할 수 없다. 현 재의 위기는 각국의 러시아 가스에 대한 의존이 과도함을 상기시키며 타국에 대한 높은 에너지공급 의존도는 국제 정세에 따라 에너지 안보에 약점이 될 수 있음을 여 실히 보여준다. 또한, 러·우 전쟁을 통해 화석 연료에 기반한 에너지 시스템이 국제 위기의 영향에 매우 취약하다는 것을 확인할 수 있다. 이와 달리 재생에너지원을 기 반으로 하는 시스템은 상대적으로 외부 환경에 대해 독립적이며, 지속가능하고 탄력 적이며, 안정적이다. 그렇기에 유럽을 비롯한 여러 국가에서 재생가능한 에너지원으 로 전환함으로써 에너지 안보 확립과 더불어 전세계 기후 위기 대처라는 더 크고 장 기적인 목표를 함께 달성하고자 한다. EU는 2022년 4월 7일부로 러시아산 석탄에 대하여 8월 이후 신규계약을 금지하는 완전한

In [28]:
filter = [
    {"term": {"metadata.source": "국제 신재생에너지 정책변화 및 시장 분석_22-26.pdf"}},
    {"term": {"metadata.type": "국제 신재생에너지 정책변화 및 시장 분석"}},
]

response = os_client.search(
    body=query_lexical(query, filter),
    index=index_name
)
print('이전에 필터를 넣치 않고 검색에 걸린 시간: ', time_took_lexical_only, 'ms')
print('지금 필터를 넣고 검색에 걸린 시간: ', response['took'], 'ms')

print("<<사용자 입력 쿼리 문장>>: ", query)

result_to_dataframe(response)

이전에 필터를 넣치 않고 검색에 걸린 시간:  5 ms
지금 필터를 넣고 검색에 걸린 시간:  10 ms
<<사용자 입력 쿼리 문장>>:  2020년 전세계 에너지 공급량은 얼마인가요?


,index_name,score,type,text
0,index-03,11.883800,국제 신재생에너지 정책변화 및 시장 분석,"제2장 국제 재생에너지 시장 동향 제2장 국제 재생에너지 시장 동향 본 연구의 주요 목적 중 하나는 매년 변화하는 재생에너지 시장 동향을 살펴보는 것으로, 자체적으로 수행한 데이터 분석 결과와 IEA, REN21, Enerdtata, BNEF, IRENA 등 주요 국제기구에서 발간하는 자료를 토대로 재생에너지 시장 변화를 살 펴보고자 한다. 먼저 1절에서는 전세계 관점에서 재생에너지 보급, 산업, 가격, 전망 에 대해 다루고, 2절에서는 국가별 재생에너지 보급 및 가격 현황을 살펴보았다. 1. 재생에너지 보급 동향 1.1. 1차에너지1) 2020년 전세계 에너지 공급량은 14,177Mtoe로 이 중 재생에너지 공급량은 14.6%에 해당하는 2,077Mtoe였다. 비중이 가장 높은 에너지원은 석유로 30.0%를 차지하였고, 다음으로 석탄(26.9%), 천연가스(23.5%), 순이다. 2010년과 비교하여 재생에너지 공급량은 508Mtoe 증가하였으며, 1차에너지에서 재생에너지 비중은 1) 여기서는 Enerdata(최종접속일: 2022.11.20)에서 제공하는 자료를 활용하여 작성하였음. 5"
1,index-03,5.876000,국제 신재생에너지 정책변화 및 시장 분석,"2.7%p 증가하였다([그림 2-1]). 재생에너지 원별로 나누어보면 2020년 기준 바이 오매스 비중이 65.1%로 가장 높고, 다음으로 수력(17.9%), 풍력(6.7%), 지열(5.3%) 태양광(3.4%), 태양열(1.6%) 순이다. 바이오매스 비중이 높은 이유는 개발도상국에 서 난방과 취사용으로 주로 활용하기 때문이다.2) 2010년과 비교하여 공급량은 바 이오매스가 191Mtoe로 가장 많이 증가하였고, 다음으로 풍력(109Mtoe), 수력 (76Mtoe), 태양광(67Mtoe) 순이다. 하지만 비중은 풍력이 4.8%p로 가장 많이 증 가하였고, 다음으로 태양광(3.2%), 지열(1.2%) 순이며, 바이오매스와 수력은 각각 8.9%p와 1.0%p 감소하였다([그림 2-2]). [그림 2-1] 에너지원별 1차에너지 공급(TPES) 비중 <2020년> <2010년> 자료: Enerdata(최종접속일: 2022.11.20)의 자료를 활용하여 저자 작성 [그림 2-2] 재생에너지 원별 1차에너지 공급(TPES) 비중 <2010년> <2020년> 자료: Enerdata(최종접속일: 2022.11.20)의 자료를 활용하여 저자 작성 2) 공지영·조일현(2021), p.4. 6"
2,index-03,5.587500,국제 신재생에너지 정책변화 및 시장 분석,"제2장 국제 재생에너지 시장 동향 2. 국가별 재생에너지 현황 비교 2.1. 국가별 재생에너지 보급 현황 [그림 2-17]은 OECD 국가별 재생에너지 발전 비중을 나타낸다. 2020년 우리나 라 재생에너지 발전 비중은 4.7% 수준으로 37위이며, OECD 평균(27.3%)과 전세계 평균(26.1%)보다 낮게 나타났다.25) 전세계 평균(26.1%)에 도달하지 못한 OECD 국 가는 우리나라를 포함하여 모두 13개국이다. OECD 중에서 재생에너지 발전 비중이 가장 높은 나라는 코스타리카(99.2%), 아이슬란드(99.1%), 노르웨이(96.1%) 순이다. 상위 10개 국가 중 덴마크를 제외한 나머지 국가는 주 에너지원이 수력이라는 특징 이 있으며, 덴마크의 주 에너지원은 풍력이다. 10위인 스웨덴의 수력 비중은 44.2%, 원자력 비중은 30.1%로 원자력 비중도 상당히 높은 것이 특징이다. 세계 각국은 자원 및 경제 상황에 따라 에너지원을 다양하게 사용하는 가운데, 영 국, 독일, 호주, 미국, 일본 등은 탄소중립 달성을 위해 에너지전환 정책을 적극적으 로 추진 중이다. 영국은 석탄 감축 정책을 펼치면서 재생에너지 보급을 빠르게 증가 시킨 나라 중 하나로, 원별로는 천연가스 비중이 36.5% 가장 높고, 다음으로 풍력 (24.2%), 원자력(16.1%), 바이오연료(11.2%) 등의 비중이 높다. 영국의 2020년 석 탄 발전 비중은 2.0% 수준으로 2015년 22.7%와 비교하여 대폭 감소하였다. 독일 은 탈원전·탈석탄을 추진 중인 국가이지만 그 결과는 미진한데, 원별 비중을 보면 석탄이 25.5%로 가장 높고, 다음으로 풍력(22.5%), 천연가스(17.1%), 원자력 (11.1%), 태양광(8.7%) 순이다. 호주는 주요 석탄 생산국으로, 원별 비중을 보면 석 탄이 54.9%로 가장 높고, 다음으로 천연가스(20.8%), 태양광(7.9%), 풍력(7.7%), 수력(5.7%) 순이다. 미국은 바이든 정부 이후 친환경 정책을 강조하는 국가로, 원별 비중을 보면 천연가스 비중이 39.3%로 가장 높고, 다음으로 석탄(20.0%), 원자력 (19.4%), 풍력(8.0%), 수력(7.4%), 태양광(2.7%) 순이다. 일본은 후쿠시마 사건 이 후 원자력 발전이 줄어들면서 대체에너지로 재생에너지 보급 추진 중인 국가로, 원 별 비중을 보면 천연가스가 37.7%로 가장 높고, 다음으로 석탄(30.4%), 수력 (8.6%), 태양광(7.6%) 순이다. 특히 일본은 에너지 섬이면서 화석연료 수입 비중이 25)OECD iLibrary홈페이지(최종접속일: 2022.9.8) 데이터 참고함. 23"
3,index-03,5.250500,국제 신재생에너지 정책변화 및 시장 분석,"높다는 점에서 우리나라와 에너지 환경은 비슷하지만, 재생에너지 비중은 우리나보 다 4배 가까이 높다. IEA 기준 우리나라의 원별 비중을 보면 석탄이 38.7%로 가장 높고, 다음으로 원자력(27.3%), 천연가스(25.8%), 태양광(3.1%), 바이오연료(1.4%) 순이다.26) [그림 2-17] OECD 국가별 재생에너지 발전 비중(2020년) 자료: OECD iLibrary홈페이지(최종접속일: 2022.9.8) 자료 활용하여 저자 작성 본고에서는 추가로 Enerdata 자료를 활용하여 전세계 186개국을 대상으로 재생에 너지 보급 수준을 비교하였다. 이를 위해 2020년 발전량 자료를 활용하였고, 사분위 수를 도출하여 국가들을 그룹화하였다. 발전량에 따른 국가들의 지리적 분포도는 [그 림 2-18]~[그림 2-21]에 제시하였다. 개별 국가가 포함된 그룹 순위에 따라 다른 색 으로 지도에 표시되어 있으며 보급 수준이 가장 높은 상위 25% 그룹(Q4)이 가장 짙 은 색으로, 보급 수준이 낮은 하위 50% 그룹(Q2, Q1)이 가장 흐린 색으로 표시되어 총 3개의 그룹을 보여준다. 26)OECD iLibrary홈페이지(최종접속일: 2022.9.8) 데이터를 참고함. 24"
4,index-03,4.405900,국제 신재생에너지 정책변화 및 시장 분석,"은 원자재 수출을 차단하고, 러시아 화폐인 루블화 결제를 강제하면서 오히려 유럽 의 심각한 에너지 수급난을 불러일으켰다.38)39) 결과적으로는 전세계 원유, 천연가 스 가격을 상승시켰을 뿐만 아니라 대체제인 석탄 가격의 수요 증가와 EU의 러시아 산 석탄수입 금지 제재안이 통과되면서 화석연료 가격이 폭등하였다.40) [그림 3-2] 국제 화석연료 가격 추이 자료: 에너지경제연구원(2022b), p.6. 러·우 전쟁이 전세계 에너지 부문에 경종을 울렸다는 사실은 부인할 수 없다. 현 재의 위기는 각국의 러시아 가스에 대한 의존이 과도함을 상기시키며 타국에 대한 높은 에너지공급 의존도는 국제 정세에 따라 에너지 안보에 약점이 될 수 있음을 여 실히 보여준다. 또한, 러·우 전쟁을 통해 화석 연료에 기반한 에너지 시스템이 국제 위기의 영향에 매우 취약하다는 것을 확인할 수 있다. 이와 달리 재생에너지원을 기 반으로 하는 시스템은 상대적으로 외부 환경에 대해 독립적이며, 지속가능하고 탄력 적이며, 안정적이다. 그렇기에 유럽을 비롯한 여러 국가에서 재생가능한 에너지원으 로 전환함으로써 에너지 안보 확립과 더불어 전세계 기후 위기 대처라는 더 크고 장 기적인 목표를 함께 달성하고자 한다. EU는 2022년 4월 7일부로 러시아산 석탄에 대하여 8월 이후 신규계약을 금지하는 완전한

## 7. 벡터 검색 (knn 검색)을 활용한 검색 - Semantic Search

- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.

In [29]:
def query_semantic(vector, filter=[], k=5):
    QUERY_TEMPLATE = {
        "size": k,
        "query": {                    
            "knn": {
                "vector_field": {
                    "vector": vector,
                    "k": k 
                }
            }           
        }
    }
    return QUERY_TEMPLATE

In [30]:
print("<<사용자 입력 쿼리 문장>>: ", query)

response = os_client.search(
    body=query_semantic(llm_emb.embed_query(query)),
    index=index_name
)
result_to_dataframe(response)

<<사용자 입력 쿼리 문장>>:  2020년 전세계 에너지 공급량은 얼마인가요?


,index_name,score,type,text
0,index-03,0.005200,국제 신재생에너지 정책변화 및 시장 분석,"참고문헌 <웹사이트> GS칼텍스 미디어허브. 2022. 러시아·우크라이나 전쟁, 에너지 안보의 무서움을 일깨우다!. https://gscaltexmediahub.com/energy/energy-column/energy_security/(최종접 속일: 2022.8.28.). 국가법령정보센터 홈페이지-파리협정. https://www.law.go.kr/trtyMInfoP.do?trtySeq=12440#:~:text=1.%20%EB%8B%B9 %EC%82%AC%EC%9E%90%EB%8A%94%20%EC%A7%80%EC%86%8D%EA%B0%80 %EB%8A%A5%ED%95%9C,%EC%A0%81%EC%9D%91%EB%AA%A9%ED%91%9C%E B%A5%BC%20%EC%88%98%EB%A6%BD%ED%95%9C%EB%8B%A4.(최종접속일: 2022.8.20). 이용성. 2022.8.11. 작성. 유럽연합, 10일부터 러시아産 석탄 수입 금지 발효. 조선비즈. https://biz.chosun.com/international/international_economy/2022/08/11/GC XBBKG5YFF7ZPFDSODV4SBQLM/(최종접속일: 2022.9.22). 정귀희. 2017.6.9. 작성. 미국 태양광 PV 산업 규모 및 태양광 발전 정책 분석. 헬로T. https://www.hellot.net/news/article.html?no=34870(최종접속일: 2022.11.20). BNEF 홈페이지. https://www.bnef.com/(최종접속일: 2022.10.20.). BNEF 홈페이지. https://www.bnef.com/interactive-datasets/2d5d59acd9000016?data-hub=13( 최종접속일: 2022.11.19.). BNEF 홈페이지. https://www.bnef.com/interactive-datasets/2d5d59acd9000047(최종접 속일: 2022.6.29.). BP Statistical Review of World Energy & Ember. https://ourworldindata.org/grapher/net-electricity-imports?country=USA~GBR ~FRA~SWE~CAN(최종접속일: 2022.11.15.). Clean Energy Wire. 2022.1.12 작성. Germany to launch emergency programme for “huge, gigantic” 2030 emissions target task. https://www.cleanenergywire.org/news/germany-launch-emergency-program me-huge-gigantic-2030-emissions-target-task(최종접속일: 2022.9.25.). 105"
1,index-03,0.004600,국제 신재생에너지 정책변화 및 시장 분석,"제2장 국제 재생에너지 시장 동향 2. 국가별 재생에너지 현황 비교 2.1. 국가별 재생에너지 보급 현황 [그림 2-17]은 OECD 국가별 재생에너지 발전 비중을 나타낸다. 2020년 우리나 라 재생에너지 발전 비중은 4.7% 수준으로 37위이며, OECD 평균(27.3%)과 전세계 평균(26.1%)보다 낮게 나타났다.25) 전세계 평균(26.1%)에 도달하지 못한 OECD 국 가는 우리나라를 포함하여 모두 13개국이다. OECD 중에서 재생에너지 발전 비중이 가장 높은 나라는 코스타리카(99.2%), 아이슬란드(99.1%), 노르웨이(96.1%) 순이다. 상위 10개 국가 중 덴마크를 제외한 나머지 국가는 주 에너지원이 수력이라는 특징 이 있으며, 덴마크의 주 에너지원은 풍력이다. 10위인 스웨덴의 수력 비중은 44.2%, 원자력 비중은 30.1%로 원자력 비중도 상당히 높은 것이 특징이다. 세계 각국은 자원 및 경제 상황에 따라 에너지원을 다양하게 사용하는 가운데, 영 국, 독일, 호주, 미국, 일본 등은 탄소중립 달성을 위해 에너지전환 정책을 적극적으 로 추진 중이다. 영국은 석탄 감축 정책을 펼치면서 재생에너지 보급을 빠르게 증가 시킨 나라 중 하나로, 원별로는 천연가스 비중이 36.5% 가장 높고, 다음으로 풍력 (24.2%), 원자력(16.1%), 바이오연료(11.2%) 등의 비중이 높다. 영국의 2020년 석 탄 발전 비중은 2.0% 수준으로 2015년 22.7%와 비교하여 대폭 감소하였다. 독일 은 탈원전·탈석탄을 추진 중인 국가이지만 그 결과는 미진한데, 원별 비중을 보면 석탄이 25.5%로 가장 높고, 다음으로 풍력(22.5%), 천연가스(17.1%), 원자력 (11.1%), 태양광(8.7%) 순이다. 호주는 주요 석탄 생산국으로, 원별 비중을 보면 석 탄이 54.9%로 가장 높고, 다음으로 천연가스(20.8%), 태양광(7.9%), 풍력(7.7%), 수력(5.7%) 순이다. 미국은 바이든 정부 이후 친환경 정책을 강조하는 국가로, 원별 비중을 보면 천연가스 비중이 39.3%로 가장 높고, 다음으로 석탄(20.0%), 원자력 (19.4%), 풍력(8.0%), 수력(7.4%), 태양광(2.7%) 순이다. 일본은 후쿠시마 사건 이 후 원자력 발전이 줄어들면서 대체에너지로 재생에너지 보급 추진 중인 국가로, 원 별 비중을 보면 천연가스가 37.7%로 가장 높고, 다음으로 석탄(30.4%), 수력 (8.6%), 태양광(7.6%) 순이다. 특히 일본은 에너지 섬이면서 화석연료 수입 비중이 25)OECD iLibrary홈페이지(최종접속일: 2022.9.8) 데이터 참고함. 23"
2,index-03,0.004500,국제 신재생에너지 정책변화 및 시장 분석,"Climate Action Tracker 홈페이지. https://climateactiontracker.org/countries/germany/net-zero-targets/(최종접속 일: 2022.9.25.). Enerdata Database. https://global-energy-data.enerdata.net/database/(최종접속일: 2022. 11. 20). Enerdata 세계에너지 및 기후 통계 –2022년 연감. https://yearbook.enerdata.co.kr/electricity/electricity-domestic-consumption- data.html(최종접속일: 2022.11.15). Energy and Climate Intelligence Unit(ECIU) 홈페이지. https://zerotracker.net/(최종접속 일: 2022. 10. 04). European Commission. 2018.7.25 작성. Joint U.S.-EU Statement following President Ju ncker's visit to the White House. https://ec.europa.eu/commission/presscorner /detail/en/STATEMENT_18_4687(최종접속일: 2022.8.25.). European Commission. 2019.3.8 작성. EU-U.S. Joint Statement: Liquefied Natural Gas (LNG) imports from the U.S. continue to rise, up by 181%. https://ec.europa.eu/ commission/presscorner/detail/en/IP_19_1531(최종접속일: 2022.8.26.). European Commission. 2022.5.18 작성. REPowerEU: A plan to rapidly reduce dependen ce on Russian fossil fuels and fast forward the green transition. https://ec.europ a.eu/commission/presscorner/detail/en/ip_22_3131(최종접속일: 2022.10.30.). European Commission. Go-to ar

이전에 어휘분석으로 검색한 결과는 아래와 같습니다. 위의 표와 비교해보시길 바랍니다.

In [31]:
print("<<사용자 입력 쿼리 문장>>: ", query)
result_to_dataframe(response_lexical_only)

<<사용자 입력 쿼리 문장>>:  2020년 전세계 에너지 공급량은 얼마인가요?


,index_name,score,type,text
0,index-03,11.883800,국제 신재생에너지 정책변화 및 시장 분석,"제2장 국제 재생에너지 시장 동향 제2장 국제 재생에너지 시장 동향 본 연구의 주요 목적 중 하나는 매년 변화하는 재생에너지 시장 동향을 살펴보는 것으로, 자체적으로 수행한 데이터 분석 결과와 IEA, REN21, Enerdtata, BNEF, IRENA 등 주요 국제기구에서 발간하는 자료를 토대로 재생에너지 시장 변화를 살 펴보고자 한다. 먼저 1절에서는 전세계 관점에서 재생에너지 보급, 산업, 가격, 전망 에 대해 다루고, 2절에서는 국가별 재생에너지 보급 및 가격 현황을 살펴보았다. 1. 재생에너지 보급 동향 1.1. 1차에너지1) 2020년 전세계 에너지 공급량은 14,177Mtoe로 이 중 재생에너지 공급량은 14.6%에 해당하는 2,077Mtoe였다. 비중이 가장 높은 에너지원은 석유로 30.0%를 차지하였고, 다음으로 석탄(26.9%), 천연가스(23.5%), 순이다. 2010년과 비교하여 재생에너지 공급량은 508Mtoe 증가하였으며, 1차에너지에서 재생에너지 비중은 1) 여기서는 Enerdata(최종접속일: 2022.11.20)에서 제공하는 자료를 활용하여 작성하였음. 5"
1,index-03,5.876000,국제 신재생에너지 정책변화 및 시장 분석,"2.7%p 증가하였다([그림 2-1]). 재생에너지 원별로 나누어보면 2020년 기준 바이 오매스 비중이 65.1%로 가장 높고, 다음으로 수력(17.9%), 풍력(6.7%), 지열(5.3%) 태양광(3.4%), 태양열(1.6%) 순이다. 바이오매스 비중이 높은 이유는 개발도상국에 서 난방과 취사용으로 주로 활용하기 때문이다.2) 2010년과 비교하여 공급량은 바 이오매스가 191Mtoe로 가장 많이 증가하였고, 다음으로 풍력(109Mtoe), 수력 (76Mtoe), 태양광(67Mtoe) 순이다. 하지만 비중은 풍력이 4.8%p로 가장 많이 증 가하였고, 다음으로 태양광(3.2%), 지열(1.2%) 순이며, 바이오매스와 수력은 각각 8.9%p와 1.0%p 감소하였다([그림 2-2]). [그림 2-1] 에너지원별 1차에너지 공급(TPES) 비중 <2020년> <2010년> 자료: Enerdata(최종접속일: 2022.11.20)의 자료를 활용하여 저자 작성 [그림 2-2] 재생에너지 원별 1차에너지 공급(TPES) 비중 <2010년> <2020년> 자료: Enerdata(최종접속일: 2022.11.20)의 자료를 활용하여 저자 작성 2) 공지영·조일현(2021), p.4. 6"
2,index-03,5.587500,국제 신재생에너지 정책변화 및 시장 분석,"제2장 국제 재생에너지 시장 동향 2. 국가별 재생에너지 현황 비교 2.1. 국가별 재생에너지 보급 현황 [그림 2-17]은 OECD 국가별 재생에너지 발전 비중을 나타낸다. 2020년 우리나 라 재생에너지 발전 비중은 4.7% 수준으로 37위이며, OECD 평균(27.3%)과 전세계 평균(26.1%)보다 낮게 나타났다.25) 전세계 평균(26.1%)에 도달하지 못한 OECD 국 가는 우리나라를 포함하여 모두 13개국이다. OECD 중에서 재생에너지 발전 비중이 가장 높은 나라는 코스타리카(99.2%), 아이슬란드(99.1%), 노르웨이(96.1%) 순이다. 상위 10개 국가 중 덴마크를 제외한 나머지 국가는 주 에너지원이 수력이라는 특징 이 있으며, 덴마크의 주 에너지원은 풍력이다. 10위인 스웨덴의 수력 비중은 44.2%, 원자력 비중은 30.1%로 원자력 비중도 상당히 높은 것이 특징이다. 세계 각국은 자원 및 경제 상황에 따라 에너지원을 다양하게 사용하는 가운데, 영 국, 독일, 호주, 미국, 일본 등은 탄소중립 달성을 위해 에너지전환 정책을 적극적으 로 추진 중이다. 영국은 석탄 감축 정책을 펼치면서 재생에너지 보급을 빠르게 증가 시킨 나라 중 하나로, 원별로는 천연가스 비중이 36.5% 가장 높고, 다음으로 풍력 (24.2%), 원자력(16.1%), 바이오연료(11.2%) 등의 비중이 높다. 영국의 2020년 석 탄 발전 비중은 2.0% 수준으로 2015년 22.7%와 비교하여 대폭 감소하였다. 독일 은 탈원전·탈석탄을 추진 중인 국가이지만 그 결과는 미진한데, 원별 비중을 보면 석탄이 25.5%로 가장 높고, 다음으로 풍력(22.5%), 천연가스(17.1%), 원자력 (11.1%), 태양광(8.7%) 순이다. 호주는 주요 석탄 생산국으로, 원별 비중을 보면 석 탄이 54.9%로 가장 높고, 다음으로 천연가스(20.8%), 태양광(7.9%), 풍력(7.7%), 수력(5.7%) 순이다. 미국은 바이든 정부 이후 친환경 정책을 강조하는 국가로, 원별 비중을 보면 천연가스 비중이 39.3%로 가장 높고, 다음으로 석탄(20.0%), 원자력 (19.4%), 풍력(8.0%), 수력(7.4%), 태양광(2.7%) 순이다. 일본은 후쿠시마 사건 이 후 원자력 발전이 줄어들면서 대체에너지로 재생에너지 보급 추진 중인 국가로, 원 별 비중을 보면 천연가스가 37.7%로 가장 높고, 다음으로 석탄(30.4%), 수력 (8.6%), 태양광(7.6%) 순이다. 특히 일본은 에너지 섬이면서 화석연료 수입 비중이 25)OECD iLibrary홈페이지(최종접속일: 2022.9.8) 데이터 참고함. 23"
3,index-03,5.250500,국제 신재생에너지 정책변화 및 시장 분석,"높다는 점에서 우리나라와 에너지 환경은 비슷하지만, 재생에너지 비중은 우리나보 다 4배 가까이 높다. IEA 기준 우리나라의 원별 비중을 보면 석탄이 38.7%로 가장 높고, 다음으로 원자력(27.3%), 천연가스(25.8%), 태양광(3.1%), 바이오연료(1.4%) 순이다.26) [그림 2-17] OECD 국가별 재생에너지 발전 비중(2020년) 자료: OECD iLibrary홈페이지(최종접속일: 2022.9.8) 자료 활용하여 저자 작성 본고에서는 추가로 Enerdata 자료를 활용하여 전세계 186개국을 대상으로 재생에 너지 보급 수준을 비교하였다. 이를 위해 2020년 발전량 자료를 활용하였고, 사분위 수를 도출하여 국가들을 그룹화하였다. 발전량에 따른 국가들의 지리적 분포도는 [그 림 2-18]~[그림 2-21]에 제시하였다. 개별 국가가 포함된 그룹 순위에 따라 다른 색 으로 지도에 표시되어 있으며 보급 수준이 가장 높은 상위 25% 그룹(Q4)이 가장 짙 은 색으로, 보급 수준이 낮은 하위 50% 그룹(Q2, Q1)이 가장 흐린 색으로 표시되어 총 3개의 그룹을 보여준다. 26)OECD iLibrary홈페이지(최종접속일: 2022.9.8) 데이터를 참고함. 24"
4,index-03,4.405900,국제 신재생에너지 정책변화 및 시장 분석,"은 원자재 수출을 차단하고, 러시아 화폐인 루블화 결제를 강제하면서 오히려 유럽 의 심각한 에너지 수급난을 불러일으켰다.38)39) 결과적으로는 전세계 원유, 천연가 스 가격을 상승시켰을 뿐만 아니라 대체제인 석탄 가격의 수요 증가와 EU의 러시아 산 석탄수입 금지 제재안이 통과되면서 화석연료 가격이 폭등하였다.40) [그림 3-2] 국제 화석연료 가격 추이 자료: 에너지경제연구원(2022b), p.6. 러·우 전쟁이 전세계 에너지 부문에 경종을 울렸다는 사실은 부인할 수 없다. 현 재의 위기는 각국의 러시아 가스에 대한 의존이 과도함을 상기시키며 타국에 대한 높은 에너지공급 의존도는 국제 정세에 따라 에너지 안보에 약점이 될 수 있음을 여 실히 보여준다. 또한, 러·우 전쟁을 통해 화석 연료에 기반한 에너지 시스템이 국제 위기의 영향에 매우 취약하다는 것을 확인할 수 있다. 이와 달리 재생에너지원을 기 반으로 하는 시스템은 상대적으로 외부 환경에 대해 독립적이며, 지속가능하고 탄력 적이며, 안정적이다. 그렇기에 유럽을 비롯한 여러 국가에서 재생가능한 에너지원으 로 전환함으로써 에너지 안보 확립과 더불어 전세계 기후 위기 대처라는 더 크고 장 기적인 목표를 함께 달성하고자 한다. EU는 2022년 4월 7일부로 러시아산 석탄에 대하여 8월 이후 신규계약을 금지하는 완전한

## 8. LangChain을 이용한 Question & Answer

- langchain의 similarity_search_with_score API를 활용하는 방법
    - [API: similarity_search_with_score](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html#langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.similarity_search)


In [32]:
from langchain.chains.question_answering import load_qa_chain

In [33]:
results = vector_db.similarity_search_with_score(
    query=query,
    k=5,
    search_type="approximate_search",
    boolean_filter={
        "bool": {
            "filter": []
        }
    }
)

In [34]:
[res[0].page_content for res in results[:3]]

['참고문헌 <웹사이트> GS칼텍스 미디어허브. 2022. 러시아·우크라이나 전쟁, 에너지 안보의 무서움을 일깨우다!. https://gscaltexmediahub.com/energy/energy-column/energy_security/(최종접 속일: 2022.8.28.). 국가법령정보센터 홈페이지-파리협정. https://www.law.go.kr/trtyMInfoP.do?trtySeq=12440#:~:text=1.%20%EB%8B%B9 %EC%82%AC%EC%9E%90%EB%8A%94%20%EC%A7%80%EC%86%8D%EA%B0%80 %EB%8A%A5%ED%95%9C,%EC%A0%81%EC%9D%91%EB%AA%A9%ED%91%9C%E B%A5%BC%20%EC%88%98%EB%A6%BD%ED%95%9C%EB%8B%A4.(최종접속일: 2022.8.20). 이용성. 2022.8.11. 작성. 유럽연합, 10일부터 러시아産 석탄 수입 금지 발효. 조선비즈. https://biz.chosun.com/international/international_economy/2022/08/11/GC XBBKG5YFF7ZPFDSODV4SBQLM/(최종접속일: 2022.9.22). 정귀희. 2017.6.9. 작성. 미국 태양광 PV 산업 규모 및 태양광 발전 정책 분석. 헬로T. https://www.hellot.net/news/article.html?no=34870(최종접속일: 2022.11.20). BNEF 홈페이지. https://www.bnef.com/(최종접속일: 2022.10.20.). BNEF 홈페이지. https://www.bnef.com/interactive-datasets/2d5d59acd9000016?data-hub=13( 최종접속일: 2022.11.19.). BNEF 홈페이지. https://www.bnef.com/interactive-datasets/2d5d59acd9000047(최종접 속일: 2022.6.29.). BP Statistical Revie

### 사용자 정의 가능한 옵션
이제 벡터 저장소가 준비되었으므로 질문을 시작할 수 있습니다.

Vector Store를 둘러싸서 LLM 입력을 받는 LangChain에서 제공하는 래퍼를 사용할 수 있습니다.
이 래퍼는 뒤에서 다음 단계를 수행합니다.
- 질문을 입력합니다.
- 질문 임베딩 생성
- 관련 문서 가져오기
- 프롬프트에 문서와 질문을 채워 넣습니다.
- 프롬프트로 모델을 호출하고 사람이 읽을 수 있는 방식으로 답변을 생성합니다.

위 시나리오에서는 질문에 대한 상황 인식 답변을 빠르고 쉽게 얻을 수 있는 방법을 탐색했습니다. 이제 문서를 가져오는 방법을 사용자 정의할 수 있는 [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/Vector_db_qa.html)의 도움으로 더 사용자 정의 가능한 옵션을 살펴보겠습니다. `chain_type` 매개변수를 사용하여 프롬프트에 추가해야 합니다. 또한 검색해야 하는 관련 문서 수를 제어하려면 아래 셀에서 'k' 매개변수를 변경하여 다른 출력을 확인하세요. 많은 시나리오에서 LLM이 답변을 생성하는 데 사용한 소스 문서가 무엇인지 알고 싶을 수 있습니다. LLM 프롬프트의 컨텍스트에 추가된 문서를 반환하는 `return_source_documents`를 사용하여 출력에서 ​​해당 문서를 가져올 수 있습니다. 'RetrievalQA'를 사용하면 모델에 특정한 사용자 정의 [프롬프트 템플릿](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html)을 제공할 수도 있습니다.

참고: 이 예에서는 Amazon Bedrock에서 LLM으로 Anthropic Claude를 사용하고 있습니다. 이 특정 모델은 입력이 'Human:' 아래에 제공되고 모델이 'Assistant:' 다음에 출력을 생성하도록 요청되는 경우 가장 잘 수행됩니다. 아래 셀에는 LLM이 기본 상태를 유지하고 컨텍스트 외부에서 응답하지 않도록 프롬프트를 제어하는 ​​방법의 예가 나와 있습니다.

#### [[REF] Using langchain for Question Answering on Own Data](https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed)

In [35]:
from langchain.schema import BaseRetriever
from typing import Any, Dict, List, Optional, List, Tuple
from langchain.callbacks.manager import CallbackManagerForRetrieverRun

# lexical(keyword) search based (using Amazon OpenSearch)
class OpenSearchLexicalSearchRetriever(BaseRetriever):
    os_client: Any
    index_name: str
    k = 3
    filter = []

    def normalize_search_results(self, search_results):
        hits = (search_results["hits"]["hits"])
        max_score = float(search_results["hits"]["max_score"])
        for hit in hits:
            hit["_score"] = float(hit["_score"]) / max_score
        search_results["hits"]["max_score"] = hits[0]["_score"]
        search_results["hits"]["hits"] = hits
        return search_results

    def update_search_params(self, **kwargs):
        self.k = kwargs.get("k", 3)
        self.filter = kwargs.get("filter", [])
        self.index_name = kwargs.get("index_name", self.index_name)

    def _reset_search_params(self, ):
        self.k = 3
        self.filter = []
        
    def query_lexical(self, query, filter=[], k=5):
        QUERY_TEMPLATE = {
            "size": k,
            "query": {
                "bool": {
                    "must": [
                        {
                            "match": {
                                "text": {
                                    "query": query,
                                    "operator":  "or"
                                }
                            }
                        }
                    ],
                    "filter": filter
                }
            }
        }
        
        if len(filter) > 0:
            QUERY_TEMPLATE["query"]["bool"]["filter"].extend(filter)
            
        return QUERY_TEMPLATE
    

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun) -> List[Document]:
        
        query = self.query_lexical(
            query=query,
            filter=self.filter,
            k=self.k
        )

        # print ("lexical search query: ")
        # print(query)
        
        search_results = self.os_client.search(
            body=query,
            index=self.index_name
        )

        results = []
        if search_results["hits"]["hits"]:
            search_results = self.normalize_search_results(search_results)
            for res in search_results["hits"]["hits"]:

                metadata = res["_source"]["metadata"]
                metadata["id"] = res["_id"]

                doc = Document(
                    page_content=res["_source"]["text"],
                    metadata=metadata
                )
                results.append((doc))

        self._reset_search_params()

        return results[:self.k]


In [36]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
# from utils.rag import run_RetrievalQA, show_context_used

In [37]:
prompt_template = """
\n\nHuman: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

\n\nAssistant:"""


PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [38]:
chain = load_qa_chain(
    llm=llm_text,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

In [39]:
boolean_filter = []
boolean_filter = [
    {"term": {"metadata.source": "국제 신재생에너지 정책변화 및 시장 분석_22-26.pdf"}},
    {"term": {"metadata.type": "국제 신재생에너지 정책변화 및 시장 분석"}},
]

In [40]:
opensearch_lexical_retriever = OpenSearchLexicalSearchRetriever(
    os_client=os_client,
    index_name=index_name,
    k=3,
    filter=boolean_filter
)

In [41]:
answer = chain.invoke(
    {
        "input_documents": opensearch_lexical_retriever.get_relevant_documents(query), 
        "question": query
    }, 
    # return_only_outputs=True
)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

제2장 국제 재생에너지 시장 동향 제2장 국제 재생에너지 시장 동향 본 연구의 주요 목적 중 하나는 매년 변화하는 재생에너지 시장 동향을 살펴보는 것으로, 자체적으로 수행한 데이터 분석 결과와 IEA, REN21, Enerdtata, BNEF, IRENA 등 주요 국제기구에서 발간하는 자료를 토대로 재생에너지 시장 변화를 살 펴보고자 한다. 먼저 1절에서는 전세계 관점에서 재생에너지 보급, 산업, 가격, 전망 에 대해 다루고, 2절에서는 국가별 재생에너지 보급 및 가격 현황을 살펴보았다. 1. 재생에너지 보급 동향 1.1. 1차에너지1) 2020년 전세계 에너지 공급량은 14,177Mtoe로 이 중 재생에너지 공급량은 14.6%에 해당하는 2,077Mtoe였다. 비중이 가장 높은 에너지원은 석유로 30.0%를 차지하였고, 다음으로 석탄(26.9%), 천연가스(23.5%), 순이다. 2010년과 비교하여 재생에너지 공급량은 508Mtoe 증가하였으며, 1차에너지에서 재생에너지 비중은 1) 여기서는 Enerdata(최종접속일: 2022.11.20)에서 제공하는 자료를 활용하여 작성하였음. 5

2.7%p 증가하였다([그림 2-1]). 재생에너지 원별로 나누어보면 2020년 기준 바이 오매스 비중이 65.1%로 가장 높고, 다음으로 수력(17.9%), 풍력(6.7%), 지열(5.3%) 태양광(3.4%), 태양열(1.6%) 순이다. 바이오

In [42]:
opensearch_semantic_retriever = vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
        "boolean_filter": boolean_filter
    }
)

In [43]:
answer = chain.invoke(
    {
        "input_documents": opensearch_semantic_retriever.get_relevant_documents(query), 
        "question": query
    }, 
    # return_only_outputs=True
)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

참고문헌 <웹사이트> GS칼텍스 미디어허브. 2022. 러시아·우크라이나 전쟁, 에너지 안보의 무서움을 일깨우다!. https://gscaltexmediahub.com/energy/energy-column/energy_security/(최종접 속일: 2022.8.28.). 국가법령정보센터 홈페이지-파리협정. https://www.law.go.kr/trtyMInfoP.do?trtySeq=12440#:~:text=1.%20%EB%8B%B9 %EC%82%AC%EC%9E%90%EB%8A%94%20%EC%A7%80%EC%86%8D%EA%B0%80 %EB%8A%A5%ED%95%9C,%EC%A0%81%EC%9D%91%EB%AA%A9%ED%91%9C%E B%A5%BC%20%EC%88%98%EB%A6%BD%ED%95%9C%EB%8B%A4.(최종접속일: 2022.8.20). 이용성. 2022.8.11. 작성. 유럽연합, 10일부터 러시아産 석탄 수입 금지 발효. 조선비즈. https://biz.chosun.com/international/international_economy/2022/08/11/GC XBBKG5YFF7ZPFDSODV4SBQLM/(최종접속일: 2022.9.22). 정귀희. 2017.6.9. 작성. 미국 태양광 PV 산업 규모 및 태양광 발전 정책 분석. 헬로T. https://www.hellot.net/news/article.html

## 9. OpenSearch Hybrid 검색

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) "Vector 서치" 하여 스코어를 얻은 후에 표준화를 하여 스코어를 구함. 
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (2) Keyword 서치도 동일하게 함.
- (3) Reciprocal Rank Fusion (RRF) 기반 Re-rank
    - Paper: https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - Desc: https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d
    - **RRF의 경우 score가 아닌 ranking 정보를 활용, 때문에 score normalization이 필요 없음**

RRF는 langchain에서 "Ensemble Retriever" 이름으로 api를 제공합니다. 
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble


### Ensemble retriever 정의
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble
- RRF 방식만 지원
- Rank constant (param "c")
    - This value determines how much influence documents in individual result sets per query have over the final ranked result set. A higher value indicates that lower ranked documents have more influence. This value must be greater than or equal to 1. Defaults to 60.
    - 숫자 높을 수록 낮은 랭크의 문서가 더 중요시 된다

In [44]:
from langchain.retrievers import EnsembleRetriever

In [45]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[opensearch_lexical_retriever, opensearch_semantic_retriever],
    weights=[0.5, 0.5],
    c=100,
    k=5
)

In [46]:
%%time
answer = chain.invoke(
    {
        "input_documents": ensemble_retriever.get_relevant_documents(query), 
        "question": query
    }
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

제2장 국제 재생에너지 시장 동향 2. 국가별 재생에너지 현황 비교 2.1. 국가별 재생에너지 보급 현황 [그림 2-17]은 OECD 국가별 재생에너지 발전 비중을 나타낸다. 2020년 우리나 라 재생에너지 발전 비중은 4.7% 수준으로 37위이며, OECD 평균(27.3%)과 전세계 평균(26.1%)보다 낮게 나타났다.25) 전세계 평균(26.1%)에 도달하지 못한 OECD 국 가는 우리나라를 포함하여 모두 13개국이다. OECD 중에서 재생에너지 발전 비중이 가장 높은 나라는 코스타리카(99.2%), 아이슬란드(99.1%), 노르웨이(96.1%) 순이다. 상위 10개 국가 중 덴마크를 제외한 나머지 국가는 주 에너지원이 수력이라는 특징 이 있으며, 덴마크의 주 에너지원은 풍력이다. 10위인 스웨덴의 수력 비중은 44.2%, 원자력 비중은 30.1%로 원자력 비중도 상당히 높은 것이 특징이다. 세계 각국은 자원 및 경제 상황에 따라 에너지원을 다양하게 사용하는 가운데, 영 국, 독일, 호주, 미국, 일본 등은 탄소중립 달성을 위해 에너지전환 정책을 적극적으 로 추진 중이다. 영국은 석탄 감축 정책을 펼치면서 재생에너지 보급을 빠르게 증가 시킨 나라 중 하나로, 원별로는 천연가스 비중이 36.5% 가장 높고, 다음으로 풍력 (24.2%), 원자력(16.1%), 바이오연료(11.2%) 등의 비중이 높다. 영국의 2020년 석 탄 발전